In [21]:
# %pip install swebench

In [7]:
from datasets import load_dataset

ds = load_dataset("princeton-nlp/SWE-bench")

In [29]:
swebench

<module 'swebench' from '/Users/wfh/code/lc/langgraph/.venv/lib/python3.11/site-packages/swebench/__init__.py'>

In [12]:
ds["dev"]

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'],
    num_rows: 225
})

In [16]:
import numpy as np

dataset = ds["dev"]
text_key = "problem_statement"  # "text"
lens = np.array(list(map(len, dataset[text_key])))
dataset = dataset.select(np.argsort(lens))

In [28]:
def format_output():
    output_dict = {"instance_id": instance_id}
    output_dict.update(basic_args)
    output_dict["text"] = f"{datum['text']}\n\n"
    response, cost = call_chat(
        output_dict["model_name_or_path"],
        output_dict["text"],
        use_azure,
        temperature,
        top_p,
    )
    completion = response.choices[0]["message"]["content"]
    total_cost += cost
    print(f"Total Cost: {total_cost:.2f}")
    output_dict["full_output"] = completion
    output_dict["model_patch"] = extract_diff(completion)
    return output_dict

In [22]:
output_dict["full_output"] = completion
output_dict["model_patch"] = extract_diff(completion)

In [ ]:
# python run_evaluation.py \
#     --predictions_path [Required] [.json]  Path to predictions \
#     --log_dir          [Required] [folder] Path to directory to save evaluation log files \
#     --swe_bench_tasks  [Required] [.json]  Path to SWE-bench task instances file \
#     --testbed          [Required] [folder] Path to testbed directory, for cloning GitHub repos to \
#     --skip_existing    [Optional] [bool]   Skip evaluating task instances with logs that already exist \
#     --timeout          [Optional] [int]    Timeout for installation + test script execution \
#     --verbose          [Optional] [bool]   Print verbosely

In [27]:
import swebench


def evaluate(
    predictions_path: str,
    swe_bench_tasks: str,
    log_dir: str,
    testbed: str,
    log_suffix: str,
    skip_existing: bool,
    timeout: int,
    verbose: bool,
    num_processes: int = -1,
):
    return swebench.run_evaluation(
        predictions_path,
        swe_bench_tasks,
        log_dir,
        testbed,
        log_suffix,
        skip_existing,
        timeout,
        verbose,
        num_processes,
    )


# def evaluate_task(run, example):